<a href="https://colab.research.google.com/github/sipocz/Colab/blob/main/Task_handout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! python -m spacy download en
! pip install wordcloud
! wget https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/master/sentiment.tsv

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
--2021-02-05 19:24:46--  https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/master/sentiment.tsv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘sentiment.tsv’

sentiment.tsv           [ <=>                ] 437.05K  --.-KB/s    in 0.1s    

2021-02-05 19:24:47 (3.09 MB/s) - ‘sentiment.tsv’ saved [447540]



# Feladat: "érzelemosztályozás" (sentiment classification)

A feladat egymondatos filmértékelések "érzelmi tartalmának" osztályozása. Csak két kategóriával dolgozunk: egy értékelés vagy negatív, vagy pozitív tartalmú.

> "Adatforrás: [UMICH SI650 - Sentiment Classification](https://www.kaggle.com/c/si650winter11/data)

> Tréning adat: 7086 sor. 
  
> Formátum: 1|0 (tab) mondat

> Az adatokat eredetileg az opinmind.com gyűjtötte össze, ami azóta megszűnt.

Az adatok a "sentiment.tsv" fájlban találhatók.

# Adatbetöltés

In [5]:
import pandas as pd

df = pd.read_csv('sentiment.tsv', sep='\t', 
                 quoting=3, # Quotes are _never_ field separators
                 header=None)

df.head()

,0,1
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...


In [6]:
df = df[[1,0]] # oszlopok átrendezése

df.rename(columns={1:"text", 0:"sentiment"}, inplace=True) # oszlopok átnevezése

df.head()

,text,sentiment
0,The Da Vinci Code book is just awesome.,1
1,this was the first clive cussler i've ever rea...,1
2,i liked the Da Vinci Code a lot.,1
3,i liked the Da Vinci Code a lot.,1
4,I liked the Da Vinci Code but it ultimatly did...,1


# Feladat: Tréning, validációs és tesztanyagra osztás

Mielőtt bármi másba fognánk (!), osszuk fel tréning, validációs és teszt részekre az adatokat.

In [7]:
# Importáljuk be a Scikitből a szükséges függvényt
from sklearn.model_selection import train_test_split

# Figyeljük meg, hogy egyszerre csak kétfelé tud osztani, így akkor járunk jól,
# ha kétszer ismételjük a hívást, "láncban"
# Ne felejtsük el fixálni a random seed-et, mondjuk 13-ra, mert az olyan szerencsés szám!:-)
df_train, df_test_valid = train_test_split(df,random_state=13)

df_test, df_valid = train_test_split(df_test_valid, random_state=13)

#assert len(df_train)==5668 and len(df_valid)==709 and len(df_test)==709
print(len(df_train), len(df_valid), len(df_test))

5314 443 1329


# Adatelemzés

In [8]:
df_train.describe()

,sentiment
count,5314.000000
mean,0.557960
std,0.496676
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


Megvizsgálhatjuk a mondatok hosszát is:

In [9]:
n_chars = df_train.text.apply(lambda x: len(x))

n_chars.describe()

count    5314.000000
mean       60.263831
std        37.885590
min        18.000000
25%        32.000000
50%        48.000000
75%        77.000000
max       203.000000
Name: text, dtype: float64

The first sentence with the maximal length:

In [10]:
long_sentence = df_train.loc[n_chars.idxmax(), "text"]
long_sentence

'A mother in Georgia wants her local school board to take Harry Potter out of the schools and libraries because, in her opinion, reading Harry Potter leads to witchcraft, which according to her is evil...'

# Extra feladat: készítsünk szófelhőt!

Vizualizáljuk együttesen és kategóriánként a mondatokat!

Eszköz: https://github.com/amueller/word_cloud


Egy jó péda: https://github.com/amueller/word_cloud/blob/master/examples/simple.py



In [11]:
# Segédfunkció a szófelhőhőz
# Input: egy _EGYSÉGES_, space-szel elválasztott string!
# Protip: https://www.tutorialspoint.com/python/string_join.htm
def do_wordcloud(text):
    from wordcloud import WordCloud
    
    # Szófelhő kép létrehozása
    wordcloud = WordCloud().generate(text)

    # Létrehozott kép megjelenítése:
    # Nyilván tehettük volt szebb helyre is az importot... :-P
    import matplotlib.pyplot as plt

    # alacsonyabb max_font_size
    wordcloud = WordCloud(max_font_size=40).generate(text)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


In [12]:
### OPCIONÁLIS FELADAT !!! ####
# Hívjuk meg a szófelhő függvényt!




In [13]:
### TASK !!! ####
# Tegyük meg ugyanezt, csak a negatív hangulatúnak besorolt mondatokkal!
# Segítség: A csak negatív hanglatú sorokat tartalmazó DataFrame alakja: (2975, 2)
# Forrás: https://pandas.pydata.org/pandas-docs/stable/indexing.html



# Bag of words (BoW) szövegreprezentáció

Minden mondatot egy olyan (ritka) fix hosszúságú vektorral fogunk jellemezni, amely a mondatban előfoduló lemmák (~ szótövek) számát tartalmazza a tréningadatban előforduló gyakori lemmákra. 

A szavakra bontásra és lemmatizálásra a nyílt forráskódú [spaCy](https://spacy.io/) Python NLP könyvtárt használjuk, amely egy egyedi lemmaazonosítókból álló listát képes előállítani a szövegekből.

In [14]:
import spacy

nlp = spacy.load("en", disable=["parser", "ner"]) 
# Csak a szófelbontásra van szülsgünk, magasabb funkciókra nem

A spaCy a szöveg nyelvészeti elemzését tartalmazó Doc objektumokat állít elő. Ezek az objektumok a szavak lemmaazonosítóit is tartalmazzák.

In [15]:
doc = nlp(long_sentence)
type(doc)

spacy.tokens.doc.Doc

In [16]:
print([token.lemma_ for token in doc ]) # Lemmák

['a', 'mother', 'in', 'Georgia', 'want', '-PRON-', 'local', 'school', 'board', 'to', 'take', 'Harry', 'Potter', 'out', 'of', 'the', 'school', 'and', 'library', 'because', ',', 'in', '-PRON-', 'opinion', ',', 'read', 'Harry', 'Potter', 'lead', 'to', 'witchcraft', ',', 'which', 'accord', 'to', '-PRON-', 'be', 'evil', '...']


In [17]:
print([token.lemma for token in doc]) # Kapcsolódó egyedi id-k

[11901859001352538922, 7963322251145911254, 3002984154512732771, 309210702643012516, 7597692042947428029, 561228191312463089, 16319852998319793599, 13293160603192985325, 14899812206273857344, 3791531372978436496, 6789454535283781228, 5164779919001708464, 2416965663249996073, 1696981056005371314, 886050111519832510, 7425985699627899538, 13293160603192985325, 2283656566040971221, 1785747669126016609, 16950148841647037698, 2593208677638477497, 3002984154512732771, 561228191312463089, 14536103007527724270, 2593208677638477497, 11792590063656742891, 5164779919001708464, 2416965663249996073, 82546335403996757, 3791531372978436496, 17905374590688478165, 2593208677638477497, 7063653163634019529, 701735504652304602, 3791531372978436496, 561228191312463089, 10382539506755952630, 15036397985088571056, 10875615029400813363]


Ezeket a lemma id listákat BoW vectorokká kell alakítanunk. Megpróbálhatnánk az egészet egyedül megírni, de szerencsére erre nincs szükség, mert a scikit-learn tartalmaz egy feature-extractort, ami pontosan ezt csinálja: ez a  [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer). Az egyszerűség kedvéért ezt használjuk.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer=lambda s: [token.lemma for token in nlp(s)], #spaCy az elemzéshez
                     min_df= 0.001) # Ignoráljuk az alacsony dolkumentum gyakoriságú lemmákat
cv


CountVectorizer(analyzer=<function <lambda> at 0x7f1bf93f1048>, binary=False,
                decode_error='strict', dtype=<class 'numpy.int64'>,
                encoding='utf-8', input='content', lowercase=True, max_df=1.0,
                max_features=None, min_df=0.001, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [19]:

sents = ["I hate this movie.", "The movie is the worst I've seen."]
bows = cv.fit_transform(sents).toarray() 
# A CountVectorizer egy ritka mátrixot produkál, így ndarray konverziót hajtujnk végre
bows

array([[1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 2, 0, 1, 1, 1, 1, 1, 1]])

A CountVectorizer segítségével az adataink text oszlopát ritka mátrixokká alakítjuk.

In [20]:
bows_train = cv.fit_transform(df_train.text)
bow_length = bows_train.shape[1]
print("BoW length:", bow_length)
bows_train

BoW length: 367


<5314x367 sparse matrix of type '<class 'numpy.int64'>'
	with 59416 stored elements in Compressed Sparse Row format>

In [21]:
bows_valid = cv.transform(df_valid.text)
bows_valid.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement
bows_test = cv.transform(df_test.text)
bows_test.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement
bows_train=cv.transform(df_train.text)
bows_train.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement



In [ ]:
bows_train.shape[1]

367

# Feladat: A modell

Felépítünk Kerasban egy egyszerű feed-forward neurális bináris osztályozó modellt, amelyet keresztentrópia veszteségfüggvénnyel tanítunk.

In [28]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

data_rows_number=bows_train.shape[0]
data_columns_number=bows_train.shape[1]



# KERAS FUNKCIONÁLIS API-T HASZNÁLUNK!


# Paraméterek
############

hidden_size = 100

# Model
#######
# Definiáljuk (példányosítsuk) az input réteget
# Adjuk meg neki a shape paraméterben a BoW reprezentáció hosszát, mint vektort
# vigyázzunk, mert a shape _EGY ELEMŰ TUPLE-T_ fogad el!!!
inputs = Input(shape=(bow_length,))

# Hidden layer
##############
# Definiáljuk a rejtett sűrű réteget, mely a fenti paraméterrel állítható méretű
# Használjunk ReLU aktivációs funkciót (bővebben később)
# Adjuk meg bemenetként az input-ot a rejtett rétegnek
# Ügyeljünk rá, hogy a Keras funkcionális API-ban a layert meghatározó paraméterek 
# a példányosítás körébe tartoznak, a réteg inputja pedig már a példány hívásához!
# (A zárójelekben van a mágia...)

hidden_output = Dense(hidden_size, activation='relu')(inputs)

# Softmax 
#########
# Definiáljuk a kimeneti Softmax réteget
# (Ami egy sűrű réteg Softmax aktivációval...)
# Emlékezzünk, pontosan két osztályunk van
# És a rejtett réteg kimenetével "etetjük"
predictions = Dense(1000, activation='softmax')(hidden_output)

# Teljes model
##############
# Nincs más hátra, mint hogy létrehozzuk a model példányt
# Bemenetre és kimenetre figyeljünk!
model = Model(inputs=inputs, outputs=predictions)

# Optimalizáló
##############
# Most fixen ezt használjuk.
optimizer = SGD(lr=0.1)
 

# Kompiláció tanítás 
####################



model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', # ezt a cross entropy variánst hasnzáljuk
                                                      # mivel az input nem one-hot enkódolt
              metrics=['accuracy']) #Training során az accuracy metirkát mérjük és printeljük.

# Tréning

In [31]:
model.fit(x=bows_train, 
          y=df_train.sentiment.values,
          validation_data=(bows_valid, df_valid.sentiment.values),
          epochs=30,
          batch_size=200)

# Kérlek, ne csak futtasd, értsd is meg!!!

Epoch 1/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0610 - accuracy: 0.9814 - val_loss: 0.0563 - val_accuracy: 0.9842
Epoch 2/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0563 - accuracy: 0.9831 - val_loss: 0.0550 - val_accuracy: 0.9842
Epoch 3/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0523 - accuracy: 0.9842 - val_loss: 0.0565 - val_accuracy: 0.9842
Epoch 4/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0490 - accuracy: 0.9849 - val_loss: 0.0482 - val_accuracy: 0.9842
Epoch 5/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0459 - accuracy: 0.9866 - val_loss: 0.0449 - val_accuracy: 0.9842
Epoch 6/30
27/27 [==============================] - 0s 4ms/step - loss: 0.0434 - accuracy: 0.9865 - val_loss: 0.0442 - val_accuracy: 0.9842
Epoch 7/30
27/27 [==============================] - 0s 10ms/step - loss: 0.0419 - accuracy: 0.9874 - val_loss: 0.0441 - val_accuracy: 0.9819
Epoch 8/30
27/27 [=

# Predikció

In [ ]:
print("=== INTERACTIVE DEMO ===")
while True:
    s = input("Enter a short text to evaluate or press return to quit: ")
    if s == "":
        break
    else:
        bow = cv.transform([s])
        prob_pred = model.predict(bow[0])
        print(f"Positive vs negative sentiment probability: {prob_pred[0,1]} vs {prob_pred[0,0]}")

=== INTERACTIVE DEMO ===
